# RAG 検索の精度向上策(advanced RAG)について

- 『LangChain と LangGraph による RAG・AI エージェント[実践]入門』より


## ・シンプルな RAG


In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.document_loaders import GitLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [30]:
### ベクトル化対象データ取得
def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

418


In [3]:
### ドキュメントのベクトル化

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# db = Chroma.from_documents(documents, embeddings, persist_directory="./chroma_db") # ベクトルストアの作成

In [4]:
# 永続化したChromaDBへの接続
# 後続のセッションや別プロセスから読み込むとき
db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
# docs = db.similarity_search("検索クエリ")


In [5]:
### シンプルRAG実装

prompt = ChatPromptTemplate.from_template(
    '''
以下の文脈だけを踏まえて質問に回答してください。

文脈："""
{context}
"""

質問：{question}
'''
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = (
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("LangChainの概要を教えて。")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、アプリケーションのライフサイクルの各段階を簡素化することを目的としています。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、チャットモデルや埋め込みモデル、ベクトルストアなど、さまざまな技術と統合されており、開発者が異なるプロバイダー間で簡単に切り替えられるように標準化されたインターフェースを提供します。また、複雑なアプリケーションの構築を支援するためのオーケストレーション機能も備えています。'

#### インデクシングの工夫について

- インデクシングについては以下の工夫がある
  - 適切な大きさでチャンク化
  - ドキュメントのカテゴリなどをメタデータ化 etc...


## 1. 検索クエリの工夫

### 1-1. HyDE (Hypothetical Document Embeddings)

- ユーザーの質問に対して LLM に仮説的な回答を推論させ、その出力を埋め込みベクトルの類似度検索に使用する。


In [ ]:
### 仮説的な回答を生成するChainの実装
hypothtical_prompt = ChatPromptTemplate.from_template(
    """
次の質問に回答する一文を書いてください。

質問：{question}
"""
)
# 仮説的な回答を生成するChain
hypothetical_chain = hypothtical_prompt | model | StrOutputParser()

### RAGのChain実装
hyde_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hypothetical_chain
        | retriever,  # 仮説的な回答を生成するChainの出力をretrieverに渡す
    }
    | prompt
    | model
    | StrOutputParser()
)

hyde_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、開発、運用、デプロイの各段階を簡素化することを目的としています。具体的には、以下のような特徴があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築することができます。\n\n2. **運用**: LangSmithを使用してアプリケーションを監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、さまざまなプロバイダーと統合し、標準インターフェースを提供することで、開発者が異なるコンポーネントを簡単に切り替えたり、組み合わせたりできるようにしています。また、複雑なアプリケーションのオーケストレーションをサポートするためのLangGraphや、アプリケーションの可視化と評価を行うLangSmithといったツールも提供しています。'

- ユーザーの質問よりも仮説的な回答のほうが埋め込みベクトルの類似度検索に適しているという想定の手法。そのため、LLM が仮説的な回答を推論しやすいケースに有効。


### 1-2. 複数の検索クエリの生成

- 複数の検索クエリを使うことで、適切なドキュメントが検索結果に含まれやすくなる可能性がある。


In [6]:
### LangChainの「with_structured_output」を使い、検索クエリのリストを生成する
from pydantic import BaseModel, Field


class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="検索クエリのリスト")


query_generation_prompt = ChatPromptTemplate.from_template(
    """
質問に対してベクターデータベースから関連文書を検索するために、３つの異なる検索クエリを生成してください。
距離ベースの類似度検索の限界を克服するために、ユーザーの質問に対して複数の視点を提供することが目標です。

質問：{question}
"""
)

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

### chain実装
multi_query_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": query_generation_chain | retriever.map(),
    }
    | prompt
    | model
    | StrOutputParser()
)

multi_query_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクトルストアなど）に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。また、複数のオープンソースライブラリで構成されており、開発者が簡単にさまざまなコンポーネントを組み合わせて使用できるように設計されています。'

- `retriever.map()`では、通常 retriever が str を受け取って list[Document]を返すのに対して、list[str]を受け取って、list[list[Document]]を返すように変換している。
- `map`は LangChain の Runnable が提供するメソッドの一つで、もとの Runnable に対して引数と戻り値を list 化するメソッド


## 2. 検索後の工夫

- https://smith.langchain.com/o/5cb6609d-ce31-51cc-962f-6052d0aff4cc/
- https://app.tavily.com/home


### 2-1. RAG-Fusion

- 複数の検索クエリを生成し、それらの検索結果を RRF で並べる RAG 手法
- 複数の検索結果の順位を融合して並べるアルゴリズムは RRF(Reciprocal Rang Fusion)を利用


In [ ]:
from langchain_core.documents import Document


def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[str]:
    # 各ドキュメントのコンテンツ（文字列）とそのスコアの対応を保持する辞書を準備
    content_score_mapping = {}

    # 検索クエリごとにループ
    for docs in retriever_outputs:
        # 検索結果のドキュメントごとにループ
        for rank, doc in enumerate(docs):
            # ドキュメントのコンテンツを取得
            content = doc.page_content

            # 初めて登場したコンテンツの場合はスコアを０で初期化
            if content not in content_score_mapping:
                content_score_mapping[content] = 0

            # (1/(順位 + k))のスコアを加算
            content_score_mapping[content] += 1 / (rank + k)

    # スコアの大きい順にソート
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)

    return [content for content, _ in ranked]

In [ ]:
rag_fusion_chain = (
    {
        "question": RunnablePassthrough(),
        "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
    }
    | prompt
    | model
    | StrOutputParser()
)

rag_fusion_chain.invoke("LangChainの概要を教えて。")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、開発、運用、デプロイの各段階を簡素化することを目的としています。\n\n### 主な特徴\n1. **標準化されたコンポーネントインターフェース**: LangChainは、さまざまなAIアプリケーションに必要なコンポーネントの標準インターフェースを提供し、異なるプロバイダー間での切り替えを容易にします。\n   \n2. **オーケストレーション**: 複数のコンポーネントやモデルを組み合わせて複雑なアプリケーションを構築するためのオーケストレーション機能を提供します。これにより、制御フローや状態管理が可能になります。\n\n3. **可観測性と評価**: LangChainは、アプリケーションの動作を監視し、迅速に評価するためのツールを提供します。これにより、開発者はアプリケーションのパフォーマンスを把握しやすくなります。\n\n### エコシステム\nLangChainは、以下のような複数のオープンソースライブラリで構成されています。\n- **langchain-core**: チャットモデルやその他のコンポーネントの基本抽象。\n- **langgraph**: LangChainコンポーネントを組み合わせて生産準備が整ったアプリケーションを構築するためのオーケストレーションフレームワーク。\n- **LangSmith**: LLMアプリケーションのトレース、監視、評価を行うためのプラットフォーム。\n\n### 使い方\nLangChainを使用することで、開発者は簡単にアプリケーションを構築し、最適化し、デプロイすることができます。具体的には、チャットボットやエージェント、情報検索システムなど、さまざまなアプリケーションを作成することが可能です。\n\nLangChainは、開発者がAIアプリケーションを迅速に構築し、運用するための強力なツールを提供します。'

### 2-2. リランクモデル

RRF では、複数の検索結果の順位を融合して並べた。別観点として、1 つの検索結果の順位についても、改めて並べ替えること（リランク）が有用な場合がある。  
検索結果を並べ替える方法の一つがリランクモデル（リランク用の機械学習モデル）を使うこと。
リランクモデルは、埋め込みベクトルの類似度検索よりも計算コストが高い代わりに、ランキングの精度が高いモデルを使用する。
<br>
ここでは、Cohere のリランクモデルを利用する。


In [14]:
from typing import Any

from langchain_cohere import CohereRerank
from langchain_core.documents import Document

In [16]:
def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:
    """
    リランクモデルを使って、検索結果を再評価し、上位のドキュメントを返す。

    Args:
        inp (dict[str, Any]): 入力データ。'documents'キーにリスト形式のDocumentが含まれる。
        top_n (int): 上位のドキュメント数。

    Returns:
        list[Document]: リランクされたドキュメントのリスト。
    """
    question = inp["question"]
    documents = inp["documents"]

    cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0", top_n=top_n)
    return cohere_reranker.compress_documents(documents=documents, query=question)


rerank_rag_chain = (
    {"question": RunnablePassthrough(), "documents": retriever}
    | RunnablePassthrough.assign(context=rerank)
    | prompt
    | model
    | StrOutputParser()
)

rerank_rag_chain.invoke("LangChainの概要を教えて。")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを作成し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、チャットモデルや埋め込みモデル、ベクトルストアなどの関連技術に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。また、複数のオープンソースライブラリで構成されており、開発者は必要なコンポーネントを選択して使用することができます。'

## 3. 複数の Retriever を使く工夫

### 3-1. LLM によるルーティング

- 質問内容により、検索対象の Retriever の使い分け
- ここでは、LangChain の公式ドキュメントの検索と Web 検索を質問内容により使い分ける RAG


In [19]:
from langchain_community.retrievers import TavilySearchAPIRetriever
from enum import Enum

In [ ]:
### 各検索器用意
# ドキュメント検索用
# LangSmithのトレースがわかりやすくなるように、with_configメソッドでrun_nameを設定
langchain_document_retriever = retriever.with_config(
    {"Run_name": "LangChain Document Retriever"}
)

# web検索用
web_retriever = TavilySearchAPIRetriever(k=3).with_config({"Run_name": "Web Retriever"})


### ユーザーの入力を元にLLMがRetrieverを選択するChain
class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"


class RouteOutput(BaseModel):
    route: Route


route_prompt = ChatPromptTemplate.from_template(
    """
質問に回答するために適切なRetrieverを選択してください。

質問：{question}
    """
)

route_chain = (
    route_prompt | model.with_structured_output(RouteOutput) | (lambda x: x.route)
)


### ルーティングの結果を踏まえて検索するroute_retriever関数と、処理全体の流れのChain(route_rag_chain)を実装
def route_retriever(inp: dict[str, Any]) -> list[Document]:
    """
    ユーザーの質問に基づいて適切なRetrieverを選択し、検索結果を返す。

    Args:
        inp (dict[str, Any]): 入力データ。'question'キーにユーザーの質問が含まれる。

    Returns:
        list[Document]: 選択されたRetrieverからの検索結果。
    """
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(
            question
        )  # 辞書ではなく、文字列を渡す！！！
    elif route == Route.web:
        return web_retriever.invoke(question)  # 辞書ではなく、文字列を渡す！！！

    raise ValueError(f"Unknown retriever: {retriever}")


route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    | RunnablePassthrough.assign(context=route_retriever)
    | prompt
    | model
    | StrOutputParser()
)

route_rag_chain.invoke("LangChainの概要を教えて。")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、アプリケーションのライフサイクルの各段階を簡素化することを目的としており、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、チャットモデルや埋め込みモデル、ベクトルストアなど、さまざまな技術と統合されており、開発者が異なるプロバイダー間で簡単に切り替えられるように標準化されたインターフェースを提供します。また、複雑なアプリケーションのオーケストレーションをサポートするために、LangGraphというライブラリも提供されています。\n\n全体として、LangChainは、AIアプリケーションの開発を容易にし、開発者が迅速に高品質なアプリケーションを構築できるようにすることを目指しています。'

In [24]:
route_rag_chain.invoke("東京の今日の天気は？")

'東京の今日の天気は曇のち雨です。昼頃から所々で雨雲が湧き、夜は広く雨になる予報です。'

### 3-2. ハイブリッド検索

- 複数の Retriever の検索結果を組み合わせて利用
- 例として、TF-IDF や BM25 によるベクトル検索を組み合わせたハイブリッド検索を構築（埋め込みベクトルの類似度検索と BM25 を使った検索の組み合わせ）


In [26]:
from langchain_community.retrievers import BM25Retriever
from langchain_core.runnables import RunnableParallel

In [ ]:
### 各retrieverを用意
# 埋め込みベクトル
chroma_retriever = retriever.with_config({"Run_name": "Chroma_Retriever"})

# BM25
# documentsは、最初の方でGitLoaderで取得したドキュメントを使用
bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"Run_name": "BM25_Retriever"}
)


### ハイブリッド検索のChainを実装
# ここでは、chroma_retrieverとbm25_retrieverの検索結果の順位をRRFで融合して並べている
hybrid_retriever = (
    RunnableParallel(
        {
            "chroma_documents": chroma_retriever,
            "bm25_documents": bm25_retriever,
        }
    )
    | (
        lambda x: [x["chroma_documents"], x["bm25_documents"]]
    )  # 並列実行の結果をリストにまとめる
    | reciprocal_rank_fusion  # RRFで順位を融合
)

hyde_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt
    | model
    | StrOutputParser()
)

hyde_rag_chain.invoke("LangChainの概要を教えて。")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化することを目的としています。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティ統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、さまざまなモデルや関連コンポーネントに対して標準化されたインターフェースを提供し、開発者がプロバイダー間で簡単に切り替えたり、コンポーネントを組み合わせたりできるようにします。また、複雑なアプリケーションのオーケストレーションをサポートし、アプリケーションの可観測性や評価を向上させるためのツールも提供しています。'